In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
import os
import cv2
import json
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
import keras
print(keras.__version__)

2.3.1


Using TensorFlow backend.


In [ ]:
input_shape = (128, 128, 3)
data_dir = "/content/gdrive/My Drive/Deepfake/Images"

real_data = [f for f in os.listdir(data_dir+'/real') if f.endswith('.png')]
fake_data = [f for f in os.listdir(data_dir+'/fake') if f.endswith('.png')]

X = []
Y = []
count_real = 0
count_fake = 0
for img in real_data:
    X.append(img_to_array(load_img(data_dir+'/real/'+img)).flatten() / 255.0)
    Y.append(1)
    count_real = count_real + 1
for img in fake_data:
    X.append(img_to_array(load_img(data_dir+'/fake/'+img)).flatten() / 255.0)
    Y.append(0)
    count_fake = count_fake + 1

Y_val_org = Y

#Normalization
X = np.array(X)
Y = to_categorical(Y, 2)

#Reshape
X = X.reshape(-1, 128, 128, 3)
print(X.shape,Y.shape)

#Train-Test split
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size = 0.1, random_state=5)

In [ ]:
#from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.layers import Conv2D,Convolution2D
from tensorflow.keras.layers import MaxPooling2D,add
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import InputLayer,BatchNormalization
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras import optimizers
#from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping


In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(128,128,3)))
model.add(BatchNormalization())
#model.add(MaxPooling2D(2,2))
model.add(Conv2D(8,(5,5),activation="relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D(2,2))
model.add(Conv2D(16,(5,5),activation="relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D(2,2))

model.add(Conv2D(16,(5,5),activation="relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D(4,4))

model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(16, activation="relu"))

model.add(Dropout(0.5))
model.add(Dense(2,activation = "softmax"))

model.compile(loss="binary_crossentropy",optimizer=optimizers.Adam(lr=1e-5, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False),metrics=['accuracy'])
model.summary();



Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 128, 128, 32)      896       
_________________________________________________________________
batch_normalization (BatchNo (None, 128, 128, 32)      128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 124, 124, 8)       6408      
_________________________________________________________________
batch_normalization_1 (Batch (None, 124, 124, 8)       32        
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 62, 62, 8)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 58, 58, 16)        3216      
_________________________________________

In [ ]:
model.fit(X_train,Y_train,batch_size=64,epochs = 500, validation_data = (X_val, Y_val), verbose = 1)







Train on 3370 samples, validate on 375 samples
Epoch 1/500
3370/3370 [==============================] - 5s 1ms/sample - loss: 0.2118 - acc: 0.8970 - val_loss: 0.2543 - val_acc: 0.9147
Epoch 2/500
3370/3370 [==============================] - 5s 1ms/sample - loss: 0.2157 - acc: 0.8929 - val_loss: 0.2578 - val_acc: 0.9173
Epoch 3/500
3370/3370 [==============================] - 5s 1ms/sample - loss: 0.2126 - acc: 0.8967 - val_loss: 0.2665 - val_acc: 0.9200
Epoch 4/500
3370/3370 [==============================] - 5s 1ms/sample - loss: 0.2193 - acc: 0.8929 - val_loss: 0.2608 - val_acc: 0.9200
Epoch 5/500
3370/3370 [==============================] - 5s 1ms/sample - loss: 0.1999 - acc: 0.8979 - val_loss: 0.2487 - val_acc: 0.9200
Epoch 6/500
3370/3370 [==============================] - 5s 1ms/sample - loss: 0.2044 - acc: 0.9039 - val_loss: 0.2581 - val_acc: 0.9200
Epoch 7/500
3370/3370 [==============================] - 5s 1ms/sample - loss: 0.2156 - acc: 0.8905 - val_loss: 0.2513 - val_acc: 0

In [ ]:
model.save("/content/gdrive/My Drive/Colab Notebooks/Deepfake/new_deepfake-detection-model_mesonet-4.h5")

In [ ]:
model.save("/content/gdrive/My Drive/Colab Notebooks/Deepfake/deepfake-detection-model_mesonet-4.h5")

In [ ]:
import dlib
import cv2
import os
import re
import json
from pylab import *
from PIL import Image, ImageChops, ImageEnhance

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')



Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
train_frame_folder = '/content/gdrive/My Drive/Colab Notebooks/Deepfake/Data/train_sample_videos'
with open(os.path.join(train_frame_folder, 'metadata.json'), 'r') as file:
    data = json.load(file)
data
'''list_of_train_data = [f for f in os.listdir(train_frame_folder) if f.endswith('.mp4')]
detector = dlib.get_frontal_face_detector()
for vid in list_of_train_data:
    count = 0
    cap = cv2.VideoCapture(os.path.join(train_frame_folder, vid))
    frameRate = cap.get(5)
    while cap.isOpened():
        frameId = cap.get(1)
        ret, frame = cap.read()
        if ret != True:
            break
        if frameId % ((int(frameRate)+1)*1) == 0:
            face_rects, scores, idx = detector.run(frame, 0)
            for i, d in enumerate(face_rects):
                x1 = d.left()
                y1 = d.top()
                x2 = d.right()
                y2 = d.bottom()
                crop_img = frame[y1:y2, x1:x2]
                if data[vid]['label'] == 'REAL':
                    cv2.imwrite('/content/gdrive/My Drive/Colab Notebooks/Deepfake/Images/real/'+vid.split('.')[0]+'_'+str(count)+'.png', cv2.resize(crop_img, (128, 128)))
                elif data[vid]['label'] == 'FAKE':
                    cv2.imwrite('/content/gdrive/My Drive/Colab Notebooks/Deepfake/Images/fake/'+vid.split('.')[0]+'_'+str(count)+'.png', cv2.resize(crop_img, (128, 128)))
                count+=1
'''

"list_of_train_data = [f for f in os.listdir(train_frame_folder) if f.endswith('.mp4')]\ndetector = dlib.get_frontal_face_detector()\nfor vid in list_of_train_data:\n    count = 0\n    cap = cv2.VideoCapture(os.path.join(train_frame_folder, vid))\n    frameRate = cap.get(5)\n    while cap.isOpened():\n        frameId = cap.get(1)\n        ret, frame = cap.read()\n        if ret != True:\n            break\n        if frameId % ((int(frameRate)+1)*1) == 0:\n            face_rects, scores, idx = detector.run(frame, 0)\n            for i, d in enumerate(face_rects):\n                x1 = d.left()\n                y1 = d.top()\n                x2 = d.right()\n                y2 = d.bottom()\n                crop_img = frame[y1:y2, x1:x2]\n                if data[vid]['label'] == 'REAL':\n                    cv2.imwrite('/content/gdrive/My Drive/Colab Notebooks/Deepfake/Images/real/'+vid.split('.')[0]+'_'+str(count)+'.png', cv2.resize(crop_img, (128, 128)))\n                elif data[vid]['

In [ ]:
print(data)

{'aagfhgtpmv.mp4': {'label': 'FAKE', 'split': 'train', 'original': 'vudstovrck.mp4'}, 'aapnvogymq.mp4': {'label': 'FAKE', 'split': 'train', 'original': 'jdubbvfswz.mp4'}, 'abarnvbtwb.mp4': {'label': 'REAL', 'split': 'train', 'original': None}, 'abofeumbvv.mp4': {'label': 'FAKE', 'split': 'train', 'original': 'atvmxvwyns.mp4'}, 'abqwwspghj.mp4': {'label': 'FAKE', 'split': 'train', 'original': 'qzimuostzz.mp4'}, 'acifjvzvpm.mp4': {'label': 'FAKE', 'split': 'train', 'original': 'kbvibjhfzo.mp4'}, 'acqfdwsrhi.mp4': {'label': 'FAKE', 'split': 'train', 'original': 'ccfoszqabv.mp4'}, 'acxnxvbsxk.mp4': {'label': 'FAKE', 'split': 'train', 'original': 'fjlyaizcwc.mp4'}, 'acxwigylke.mp4': {'label': 'FAKE', 'split': 'train', 'original': 'ffcwhpnpuw.mp4'}, 'aczrgyricp.mp4': {'label': 'FAKE', 'split': 'train', 'original': 'slwkmefgde.mp4'}, 'adhsbajydo.mp4': {'label': 'FAKE', 'split': 'train', 'original': 'fysyrqfguw.mp4'}, 'adohikbdaz.mp4': {'label': 'FAKE', 'split': 'train', 'original': 'qjlhemtkx